In [ ]:
#-----------------code start----------------#

import csv
import numpy as np
from glob import glob
import os
import shutil
const_empty = np.zeros(shape = (512,640))
iter_empty = np.zeros(shape = (512,640))
def Wcloud(variable_const,variable_iter,CTH,CBH):
    inp = open("UVSPEC_REPTRAN_THERMAL.INP","w+") 
    inp.write("data_files_path /opt/libradtran/2.0.2/share/libRadtran/data\r\n")
    inp.write("atmosphere_file /home/sshohan/libRadtran/AFGLUS.70KM\r\n")
    inp.write("rte_solver disort\r\n")
    inp.write("pseudospherical\r\n")
    inp.write("mol_abs_param reptran fine\r\n")
    inp.write("output_quantity brightness\r\n")
    inp.write("radiosonde /home/sshohan/libRadtran/dropsonde_D20200202_180218_PQC.txt H2O RH\r\n")
    inp.write("source thermal\r\n")
    inp.write("wavelength 7700 12000\r\n")
    inp.write("time 2020 02 02 18 02 18")
    inp.write("latitude N 14 29 85")
    inp.write("longitude W 57 67 115")
    inp.write("zout 10.32\r\n") #altitude
    inp.write("filter_function_file /home/sshohan/libRadtran/Filter_function.txt normalize\r\n")
    inp.write("wc_properties mie interpolate\r\n")
    inp.write("wc_file 1D  /home/sshohan/libRadtran_LUT/wc_mp.dat\r\n")
    inp.write("umu 1.0\r\n") 
    inp.write("output_process integrate\r\n")
    inp.write("output_user lambda uu\r\n")
    inp.close()
    for i in variable_const:
        for j in variable_iter:
            with open("wc_mp.dat","w+") as file:
                csv_writer = csv.writer(file, delimiter = " ")
                csv_writer.writerow(['#alt(km)','LWC(g/m3)','r_eff(um)'])
                csv_writer.writerow([CTH,0,0])
                csv_writer.writerow([CBH,j,i]) ###ONLY CHANGE### switch order between(i and j) according to variables_const and variables_iter
            os.popen('uvspec <UVSPEC_REPTRAN_THERMAL.INP> TEST_REPTRAN_THERMAL.out').read()
            output_BT = (np.loadtxt('TEST_REPTRAN_THERMAL.out')).round(2)
            data = (np.loadtxt('Test_filter2_org.txt')).round(2)
            match_index = np.argwhere(data == output_BT[1])
            for x,y in match_index:
                try:
                    if const_empty[x,y] == 0 and iter_empty[x,y] == 0:
                        const_empty[x,y] = i
                        iter_empty[x,y] = j
                    else:
                        continue
                except:
                    SyntaxError
            np.savetxt('const_filled.txt',const_empty,fmt = '%.2f')
            np.savetxt('iter_filled.txt',iter_empty,fmt = '%.2f')       
variable_const = np.linspace(4,30,num = 100).round(decimals = 2) ## select variable which will be constant for each iteration 
variable_iter = np.linspace(0.1,1.5,num = 100).round(decimals = 2) ##select variable for iteration 
CTH = 1 ## cloud top height
CBH = 0.5 ## cloud base height
Wcloud(variable_const,variable_iter,CTH,CBH)

#-----------------code end----------------#